In [34]:
import numpy as np
import math
from numpy import linalg as la
# math.isclose(0.1 + 0.2, 0.3)
np.set_printoptions(suppress=True, linewidth=np.nan)


### Utilities

In [35]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[c.reshape(1, -1), A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [36]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = np.Inf
    for k in range(len(b)):
        if col[k] > 0:
            if min_val > (min_val_temp := b[k] / col[k]):
                min_val = min_val_temp
                r_min = k
    return r_min


In [37]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row] = tableau[row]/tableau[row, col]
    # pivot proccess
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col]*tableau[row, :]
    return tableau


### Simplex Algorithm

In [38]:
def simplex(tableau: np.ndarray, basic_var: list[int]) -> None:
    print(f"x_B = {basic_var}\n{tableau}")
    while (tableau[0] > 0).any():
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(max_col=tableau[1:, c_max], b=tableau[1:, -1]) + 1
        # pivoting
        tableau = pivoting(tableau, r_min, c_max)
        # swap row with col
        basic_var[r_min - 1] = c_max
        print(f"x_B = {basic_var}\n{tableau}")


### Big-M
The big-M procedure, each such constraint $i$ is augmented, together with its slack variable, with a so-called artificial variable $u_i$,  and the objective function is augmented with $−Mu_i$, where $M$ is a big positive real number. For big values of $M$ the simplex algorithm will put highest priority on making the value of the factor $Mu_i$ as small as possible, thereby setting the value of $u_i$ equal to zero. Big-M and two-phases are used when $0$ isn't feasible basic solution.

In [45]:
def big_M(tableau: np.ndarray, basic_var: list[int], artificial_var: list[int]):
    print(f"x_B = {basic_var}\n{tableau}")
    # correct first row of tableau for each artificial var because the artificial_var position is -M
    for u in artificial_var:
        row = minimum_ratio_test(max_col=tableau[1:, u], b=np.ones_like(basic_var)) + 1
        tableau[row] = tableau[row] / tableau[row, u]
        tableau[0] = tableau[0] - tableau[0, u] * tableau[row, :]
    
    simplex(tableau, basic_var)


### Two-Phases
The two-phase procedure add, in the same manner as employed in the big-M procedure, but instead of adding each artificial variable to the objetive with a large negative coefficient, the objetive function is replace by minus the sum of all artificial variables . During the **first phase**, the simplex algorithm tries to maximize this objetive, effectively trying to give all artificial variables the value zero.  By doing this a feasible basic solution is founded. When this has happened, the **second-phase** starts by replacing the artificial objetive function by the objetive function of the original model and solve this with simplex method.

In [40]:
def two_phases(tableau: np.ndarray, c: np.ndarray, basic_var: list[int], artificial_var: list[int]):
    print(f"x_B = {basic_var}\n{tableau}")
    # correct first row of tableau because the artificial_var position is -1
    for u in artificial_var:
        row = minimum_ratio_test(max_col=tableau[1:, u], b=np.ones_like(basic_var)) + 1
        tableau[row] = tableau[row] / tableau[row, u]
        tableau[0] = tableau[0] - tableau[0, u] * tableau[row, :]

    print(f"x_B = {basic_var}\n{tableau}")

    # phase one
    # while until each artifical var be negative and not be basic var
    while all(((u in basic_var) or (tableau[0, u] > 0)) for u in artificial_var):
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(max_col=tableau[1:, c_max], b=tableau[1:, -1]) + 1
        # pivoting
        tableau = pivoting(tableau, r_min, c_max)
        # swap row with col
        basic_var[r_min - 1] = c_max
        print(f"x_B = {basic_var}\n{tableau}")

    # phase two
    # delete the artificial variable because we found a solution factible
    tableau = np.delete(tableau, artificial_var, axis=1)
    # put c in first row of tableau
    tableau[0, :len(c)] = c
    # solve with simplex
    simplex(tableau, basic_var)


### Examples

**Simplex**: Model Dovetail

In [41]:
A = np.array([[1, 1, 1, 0, 0, 0],
              [3, 1, 0, 1, 0, 0],
              [1, 0, 0, 0, 1, 0],
              [0, 1, 0, 0, 0, 1.0]])

b = np.array([9, 18, 7, 6.])

c = np.array([3, 2, 0, 0, 0, 0.])

tableau = build_tableau(A, b, c)
basic_var = [2, 3, 4, 5]
simplex(tableau, basic_var);


x_B = [2, 3, 4, 5]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
x_B = [2, 0, 4, 5]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
x_B = [1, 0, 4, 5]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]


**BIG-M**:  Ejemplo visto el ciclo pasado Lineal 6 

In [46]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1", 6: "u2"}
A = np.array([[2, -1, -1, 0, 0, 1, 0],
              [-1, 2, 0, -1, 0, 0, 1],
              [1, 1, 0, 0, 1, 0, 0.]])

b = np.array([4, 2, 12.])
M = 10
c = np.array([2, 1, 0, 0, 0, -M, -M])

tableau = build_tableau(A, b, c)

basic_var = [4, 5, 6]
artificial_var = [5, 6]
big_M(tableau, basic_var, artificial_var)  # FIN NO ACOTADO


x_B = [4, 5, 6]
[[  2.   1.   0.   0.   0. -10. -10.   0.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [4, 5, 6]
[[ 12.  11. -10. -10.   0.   0.   0.  60.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [0, 5, 6]
[[  0.   17.   -4.  -10.    0.   -6.    0.   36. ]
 [  1.   -0.5  -0.5   0.    0.    0.5   0.    2. ]
 [  0.    1.5  -0.5  -1.    0.    0.5   1.    4. ]
 [  0.    1.5   0.5   0.    1.   -0.5   0.   10. ]]
x_B = [0, 1, 6]
[[  0.           0.           1.66666667   1.33333333   0.         -11.66666667 -11.33333333  -9.33333333]
 [  1.           0.          -0.66666667  -0.33333333   0.           0.66666667   0.33333333   3.33333333]
 [  0.           1.          -0.33333333  -0.66666667   0.           0.33333333   0.66666667   2.66666667]
 [  0.           0.           1.           1.           1.      

**Two-Phases**: Dovetail Model modified

In [48]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "x6", 6: "x7", 7: "u1"}

tableau = np.array([[0, 0, 0, 0, 0, 0, 0, -1, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 9.],
                    [3, 1, 0, 1, 0, 0, 0, 0, 18],
                    [1, 0, 0, 0, 1, 0, 0, 0, 7.],
                    [0, 1, 0, 0, 0, 1, 0, 0, 6.],
                    [1, 1, 0, 0, 0, 0, -1, 1, 5]])

c = np.array([3, 2])

basic_var = [2, 3, 4, 5, 7]

artificial_var = [7]

two_phases(tableau, c, basic_var, artificial_var) 


x_B = [2, 3, 4, 5, 7]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
x_B = [2, 3, 4, 5, 7]
[[ 1.  1.  0.  0.  0.  0. -1.  0.  5.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
x_B = [2, 3, 4, 5, 0]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  1.  0.  0.  0.  1. -1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3. -3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1. -1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
x_B = [2, 3, 4, 5, 0]
[[ 3.  2.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  5